In [54]:
import numpy as np
import random 

In [55]:
# ----------------------------
# 0) Utilities
# ----------------------------
def _rng(seed):
    return np.random.default_rng(seed)

def pad_to_pmax(a_time_p, p_max):
    """Pad [T, p_true] -> [T, p_max] with zeros."""
    T, p = a_time_p.shape
    out = np.zeros((T, p_max), dtype=a_time_p.dtype)
    out[:, :p] = a_time_p
    return out

def enforce_l1_stability(a_time, l1_cap=0.95, eps=1e-12):
    """
    Simple, robust stability-ish heuristic for AR:
      scale coefficients at each time t so sum_k |a_k(t)| <= l1_cap
    This is sufficient to prevent most explosions for benchmarking.
    """
    l1 = np.sum(np.abs(a_time), axis=1, keepdims=True)
    scale = np.minimum(1.0, l1_cap / (l1 + eps))
    return a_time * scale

In [56]:
# ----------------------------
# 1) Smooth coefficient schedules (Task 1)
# Each returns a_time with shape [T, p_true]
# ----------------------------

def schedule_sinusoid(T, p, rng, freq_range=(1/8000, 1/2000), amp_range=(0.05, 0.25)):
    t = np.arange(T)
    freqs  = rng.uniform(freq_range[0], freq_range[1], size=p)
    phases = rng.uniform(0, 2*np.pi, size=p)
    amps   = rng.uniform(amp_range[0], amp_range[1], size=p)
    a = np.zeros((T, p), dtype=np.float32)
    for k in range(p):
        a[:, k] = amps[k] * np.sin(2*np.pi*freqs[k]*t + phases[k])
    return a

def schedule_fourier(T, p, rng, M=3, base_freq_range=(1/9000, 1/2500), amp_scale=0.18):
    t = np.arange(T)
    base_f = rng.uniform(base_freq_range[0], base_freq_range[1], size=p)
    a = np.zeros((T, p), dtype=np.float32)
    for k in range(p):
        # Random harmonic amplitudes decay with m
        for m in range(1, M+1):
            A = rng.normal(0, amp_scale/(m**1.2))
            phi = rng.uniform(0, 2*np.pi)
            a[:, k] += A * np.sin(2*np.pi*(m*base_f[k])*t + phi)
    return a

def schedule_quasiperiodic(T, p, rng, f1_range=(1/9000, 1/3000), f2_range=(1/7000, 1/2500), amp_range=(0.05, 0.18)):
    t = np.arange(T)
    f1 = rng.uniform(f1_range[0], f1_range[1], size=p)
    f2 = rng.uniform(f2_range[0], f2_range[1], size=p)
    A1 = rng.uniform(amp_range[0], amp_range[1], size=p)
    A2 = rng.uniform(amp_range[0], amp_range[1], size=p)
    ph1 = rng.uniform(0, 2*np.pi, size=p)
    ph2 = rng.uniform(0, 2*np.pi, size=p)
    a = np.zeros((T, p), dtype=np.float32)
    for k in range(p):
        a[:, k] = A1[k]*np.sin(2*np.pi*f1[k]*t + ph1[k]) + A2[k]*np.sin(2*np.pi*f2[k]*t + ph2[k])
    return a

def schedule_polynomial_drift(T, p, rng, degree=2, coef_scale=0.20):
    # use normalized time in [-1, 1]
    tn = np.linspace(-1, 1, T)
    a = np.zeros((T, p), dtype=np.float32)
    for k in range(p):
        # small polynomial coefficients
        betas = rng.normal(0, coef_scale, size=degree+1)
        # a_k(t) = sum_d beta_d * tn^d
        ak = np.zeros(T)
        for d in range(degree+1):
            ak += betas[d] * (tn**d)
        a[:, k] = ak
    return a

def schedule_logistic_transition(T, p, rng, amp_range=(0.05, 0.25), tau_range=(300, 1500)):
    t = np.arange(T)
    a = np.zeros((T, p), dtype=np.float32)
    for k in range(p):
        alo = rng.uniform(-amp_range[1], amp_range[1])
        ahi = rng.uniform(-amp_range[1], amp_range[1])
        t0  = rng.integers(int(0.2*T), int(0.8*T))
        tau = rng.uniform(tau_range[0], tau_range[1])
        s = 1.0 / (1.0 + np.exp(-(t - t0)/tau))
        a[:, k] = alo + (ahi - alo) * s
    return a

def schedule_multi_sigmoid(T, p, rng, J=3, amp_step=0.12, tau_range=(200, 1200)):
    t = np.arange(T)
    a = np.zeros((T, p), dtype=np.float32)
    for k in range(p):
        base = rng.uniform(-0.05, 0.05)
        ak = base * np.ones(T)
        # multiple smooth steps
        t0s = np.sort(rng.integers(int(0.1*T), int(0.9*T), size=J))
        for j in range(J):
            delta = rng.normal(0, amp_step)
            tau = rng.uniform(tau_range[0], tau_range[1])
            s = 1.0 / (1.0 + np.exp(-(t - t0s[j])/tau))
            ak += delta * s
        a[:, k] = ak
    return a

def schedule_gaussian_bumps(T, p, rng, J=4, amp_range=(0.05, 0.22), width_range=(200, 1200)):
    t = np.arange(T)
    a = np.zeros((T, p), dtype=np.float32)
    for k in range(p):
        base = rng.uniform(-0.03, 0.03)
        ak = base * np.ones(T)
        for _ in range(J):
            mu = rng.uniform(0.1*T, 0.9*T)
            sig = rng.uniform(width_range[0], width_range[1])
            c = rng.uniform(-amp_range[1], amp_range[1])
            ak += c * np.exp(-0.5 * ((t - mu)/sig)**2)
        a[:, k] = ak
    return a

def schedule_smooth_random(T, p, rng, smooth_sigma=250, amp=0.20):
    """
    GP-like smooth random function per lag (via Gaussian smoothing in frequency domain).
    Produces a very different, non-periodic but smooth class.
    """
    a = np.zeros((T, p), dtype=np.float32)
    # build Gaussian kernel in frequency domain for convolution
    freqs = np.fft.rfftfreq(T)
    # Gaussian smoothing kernel: exp(-2*pi^2*sigma^2*f^2)
    H = np.exp(-2*(np.pi**2) * (smooth_sigma**2) * (freqs**2))
    for k in range(p):
        w = rng.normal(0, 1, size=T)
        W = np.fft.rfft(w)
        sm = np.fft.irfft(W * H, n=T)
        sm = sm / (np.std(sm) + 1e-8)
        a[:, k] = (amp * sm).astype(np.float32)
    return a

# Registry of families you can benchmark
SCHEDULES = {
    "sinusoid": schedule_sinusoid,
    "fourier": schedule_fourier,
    "quasiperiodic": schedule_quasiperiodic,
    "poly_drift": schedule_polynomial_drift,
    "logistic": schedule_logistic_transition,
    "multi_sigmoid": schedule_multi_sigmoid,
    "gaussian_bumps": schedule_gaussian_bumps,
    "smooth_random": schedule_smooth_random,
}


In [57]:
# ----------------------------
# 2) Generic TVAR simulator (Task 1 core)
# ----------------------------
def simulate_tvar(a_time, noise_std=0.3, seed=0, burn_in=0):
    """
    Simulate a TVAR process:
        x_t = a(t)^T [x_{t-1}, ..., x_{t-p}] + eps_t

    Args
    ----
    a_time : array, shape [T, p]
        Time-varying coefficients for each lag.
        a_time[t] = [a1(t), ..., ap(t)].
    noise_std : float
        Std dev of Gaussian innovations eps_t.
    seed : int
        RNG seed.
    burn_in : int
        Number of initial simulated samples to discard.

    Returns
    -------
    x : array, shape [T]
        Simulated time series after discarding burn-in.
    """
    rng = _rng(seed)
    T, p = a_time.shape

    # Simulate a longer series, then drop the first burn_in samples
    T_full = T + int(burn_in)
    x_full = np.zeros(T_full, dtype=np.float32)
    eps = rng.normal(0.0, noise_std, size=T_full).astype(np.float32)

    # Initialize first p values
    x_full[:p] = eps[:p]

    # Main loop: for t >= p, use the same a_time indexing once we pass burn-in
    # We want the kept segment (after burn-in) to correspond to a_time[0..T-1].
    for t in range(p, T_full):
        # Map full-time index t -> coefficient index tau in [0, T-1]
        tau = t - int(burn_in)
        if tau < 0:
            # During burn-in: hold coefficients at the first value (a_time[0])
            a_t = a_time[0]
        else:
            a_t = a_time[tau]

        lags = x_full[t-p:t][::-1]  # [x_{t-1},...,x_{t-p}]
        x_full[t] = float(np.dot(a_t, lags) + eps[t])

    # Drop burn-in
    x = x_full[int(burn_in):]
    return x

In [58]:
# ----------------------------
# 3) One-sample generator + dataset wrapper (Task 2 groundwork)
# ----------------------------

def generate_one_tvar_sample(
    T=10_000,
    p_max=6,
    schedule_name="sinusoid",
    noise_std=0.3,
    seed=0,
    l1_cap=0.95
):
    rng = _rng(seed)
    p_candidates = np.array([1, 2, 4, 6], dtype=int)
    p_true = int(rng.choice(p_candidates))

    # coefficient schedule for true p
    a_tp = SCHEDULES[schedule_name](T=T, p=p_true, rng=rng)
    a_tp = enforce_l1_stability(a_tp, l1_cap=l1_cap)

    # simulate signal
    x = simulate_tvar(a_tp, noise_std=noise_std, seed=seed+123)

    # pad coeffs to p_max for consistent storage
    a_t = pad_to_pmax(a_tp.astype(np.float32), p_max)

    meta = {
        "T": T,
        "p_max": p_max,
        "p_true": p_true,
        "schedule": schedule_name,
        "noise_std": float(noise_std),
        "seed": int(seed),
        "l1_cap": float(l1_cap),
        "p_candidates": [1, 2, 4, 6],
    }
    return x, a_t, meta

def generate_dataset(
    n_per_class=10,
    classes=None,
    T=1000,
    noise_range=(0.1, 0.6),
    seed=0,
    dtype=np.float32
):
    
    p_max = 10

    if classes is None:
        classes = list(SCHEDULES.keys())

    rng = _rng(seed)
    N = n_per_class * len(classes)

    X = np.zeros((N, T), dtype=dtype)
    A = np.zeros((N, T, p_max), dtype=dtype)
    p_true = np.zeros((N,), dtype=np.int32)
    class_id = np.zeros((N,), dtype=np.int32)
    noise_std = np.zeros((N,), dtype=dtype)

    class_names = list(classes)

    metas = []
    idx = 0
    for cid, cname in enumerate(class_names):
        for _ in range(n_per_class):
            s = int(rng.integers(0, 2**31 - 1))
            ns = float(rng.uniform(noise_range[0], noise_range[1]))
            x, a_t, meta = generate_one_tvar_sample(
                T=T, p_max=p_max, schedule_name=cname, noise_std=ns, seed=s
            )
            X[idx] = x.astype(dtype)
            A[idx] = a_t.astype(dtype)
            p_true[idx] = meta["p_true"]
            class_id[idx] = cid
            noise_std[idx] = ns
            metas.append(meta)
            idx += 1

    dataset = {
        "X": X,
        "A": A,
        "p_true": p_true,
        "class_id": class_id,
        "class_names": np.array(class_names),
        "noise_std": noise_std,
        "meta": metas,  # Python list of dicts; fine for notebook use
    }
    return dataset

def save_dataset_npz(path, dataset):
    # Save large arrays in npz; meta dicts as object array
    np.savez_compressed(
        path,
        X=dataset["X"],
        A=dataset["A"],
        p_true=dataset["p_true"],
        class_id=dataset["class_id"],
        class_names=dataset["class_names"],
        noise_std=dataset["noise_std"],
        meta=np.array(dataset["meta"], dtype=object),
    )

def load_dataset_npz(path):
    d = np.load(path, allow_pickle=True)
    return {
        "X": d["X"],
        "A": d["A"],
        "p_true": d["p_true"],
        "class_id": d["class_id"],
        "class_names": d["class_names"],
        "noise_std": d["noise_std"],
        "meta": list(d["meta"]),
    }

In [59]:
# import matplotlib.pyplot as plt

# T = 1000
# p_max = 20

# x, a_t, meta = generate_one_tvar_sample(
#     T=T, p_max=p_max,
#     schedule_name="poly_drift",
#     noise_std=0.3,
#     seed=42
# )

# print(meta)

# plt.figure(figsize=(12,6))
# plt.subplot(2,1,1)
# plt.plot(x, alpha=0.8)
# plt.title("TVAR signal x[t]")
# plt.grid(True, alpha=0.3)

# plt.subplot(2,1,2)
# p_true = meta["p_true"]
# for k in range(p_true):
#     plt.plot(a_t[:,k], label=f"a{k+1}(t)")
# plt.title("True coefficients a_k(t) for k<=p_true")
# plt.legend(ncol=4, fontsize=8)
# plt.grid(True, alpha=0.3)

# plt.tight_layout()
# plt.show()

In [60]:
# 8 families
classes = list(SCHEDULES.keys())

# small pilot: 10 per family => 80 samples total
pilot = generate_dataset(
    n_per_class=10,
    classes=classes,
    T=10_000,
    noise_range=(0.2, 0.5),
    seed=123,
    dtype=np.float32
)

print("X:", pilot["X"].shape, "A:", pilot["A"].shape)
print("class_names:", pilot["class_names"])
print("p_true distribution:", np.unique(pilot["p_true"], return_counts=True))

X: (80, 10000) A: (80, 10000, 10)
class_names: ['sinusoid' 'fourier' 'quasiperiodic' 'poly_drift' 'logistic'
 'multi_sigmoid' 'gaussian_bumps' 'smooth_random']
p_true distribution: (array([1, 2, 4, 6], dtype=int32), array([17, 19, 29, 15]))


In [61]:
save_dataset_npz("tvar_pilot_T10000.npz", pilot)